In [6]:
from numpy.linalg import norm
from tqdm import tqdm
import csv
import numpy as np
from scipy.sparse import spdiags
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def row_normalize(A):
    '''
    Perform row-normalization of the given matrix
    inputs
        A : matrix
        (n x n) input matrix where n is # of nodes
    outputs
        nA : matrix
        (n x n) row-normalized matrix
    '''
    n = A.shape[0]

    d = A.sum(axis=1)
    d = np.asarray(d).flatten()

    # handle 0 entries in d
    d = np.maximum(d, np.ones(n))
    invd = 1.0 / d

    invD = spdiags(invd, 0, n, n)

    # compute row normalized adjacency matrix by nA = invD * A
    nA = invD.dot(A)

    return nA

def iterate(A, q, c=0.3, epsilon=1e-20,
            max_iters=100, handles_deadend=True, norm_type=1):
    """
    Perform power iteration for Personalized PageRank
    inputs
        A : matrix
            input matrix
        q : ndarray
            query vector
        c : float
            restart probability
        epsilon : float
            error tolerance for power iteration
        max_iters : int
            maximum number of iterations for power iteration
        handles_deadend : bool
            if true, it will handle the deadend issue in power iteration
            otherwise, it won't, i.e., no guarantee for sum of RWR scores
            to be 1 in directed graphs
        norm_type : int
            type of norm used in measuring residual at each iteration
    outputs
        x : ndarray
            result vector
    """
    x = q
    old_x = q
    residuals = np.zeros(max_iters)

    pbar = tqdm(total=max_iters)
    for i in range(max_iters):
        if handles_deadend:
            x = (1 - c) * (A.dot(old_x))
            S = np.sum(x)
            x = x + (1 - S) * q
        else:
            x = (1 - c) * (A.dot(old_x)) + (c * q)

        residuals[i] = norm(x - old_x, norm_type)
        pbar.set_description("Residual at %d-iter: %e" % (i, residuals[i]))

        if residuals[i] <= epsilon:
            pbar.set_description("The iteration has converged at %d-iter" % (i))
            #  pbar.update(max_iters)
            break

        old_x = x
        pbar.update(1)

    pbar.close()

    return x, residuals[0:i + 1]

In [7]:
adj=np.load('adjacent_matrix_weighted.npy')
N=len(adj)
fea=np.load('feature_matrix_basic_full.npy')
blacklist=fea[:,0]==1
seeds=np.argwhere(np.logical_or(fea[:,3]>3,fea[:,0]==1))
seeds=np.concatenate(seeds).tolist()
q=np.zeros(N)
q[seeds] = 1.0 / len(seeds)
norm_adj=row_normalize(adj)
result=iterate(norm_adj,q,c=0.5, epsilon=1e-40,
            max_iters=1000, handles_deadend=True, norm_type=1)
score=result[0].tolist()

Residual at 999-iter: 6.973506e-17: 100%|██████████| 1000/1000 [11:24<00:00,  1.36it/s]


In [9]:
csvfile=open("score3_weighted.csv", "w")
w = csv.writer(csvfile)
j=0
for i in score:
    w.writerow([j,i])
    j+=1
csvfile.close()

In [49]:
adj

array([[0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0.6, 0. , 0.3],
       [0. , 0. , 0. , ..., 0.9, 0. , 0.6],
       ...,
       [0. , 0.6, 0.9, ..., 0. , 0.6, 0.6],
       [0. , 0. , 0. , ..., 0.6, 0. , 0. ],
       [0. , 0.3, 0.6, ..., 0.6, 0. , 0. ]])